In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelBinarizer,OneHotEncoder,LabelEncoder
# 设置jupyter数据框最大显示的列数
pd.set_option('max_columns',100)
from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
mpl.rcParams['axes.titlesize'] = 16
from dateutil.parser import parse

In [35]:
# 读取数据
train = pd.read_csv('../d_train_20180102.csv',encoding='gb2312',index_col='id')
test = pd.read_csv('../d_test_A_20180102.csv',encoding='gb2312',index_col='id')
test_real_y = pd.read_csv('../d_answer_a_20180128.csv',encoding='gb2312',header=None)
test_b = pd.read_csv('../d_test_B_20180128.csv',encoding='gb2312',index_col='id')
# train = data_train.copy()
# test = data_test.copy()

# 把血糖单独拿出来
y = train['血糖']
train.drop('血糖',axis=1,inplace=True)
data = pd.concat([train,test])
combined = pd.concat([data,test_b])
train_index ,test_index = data.index, test_b.index
combined_copy = combined.fillna(combined.median(axis=0))
# features_drop = ['体检日期','乙肝表面抗原','乙肝表面抗体','乙肝e抗原','乙肝e抗体','乙肝核心抗体']

# train.drop(features_drop,axis=1,inplace=True)
# test.drop(features_drop,axis=1,inplace=True)

In [3]:
combined.shape

(7642, 40)

In [75]:
# combined_cut.head()

In [36]:
# 连续特征分箱 # 按照列名的顺序
# 不算 '乙肝表面抗原','乙肝表面抗体','乙肝e抗原','乙肝e抗体','乙肝核心抗体'
cut_interval = [[-np.inf,15,40,np.inf],[-1,6,24,np.inf],[-1,45,135,np.inf],[-1,50,np.inf],[-1,60,85,np.inf],[-1,35,51,np.inf],
                [-1,20,30,np.inf],[-1,1.5,2.5,np.inf],[-1,0.45,1.69,np.inf],[-1,2.85,5.69,np.inf],[-1,1.4,np.inf],[-1,2,4,np.inf],
               [-1,2,7.1,np.inf],[-1,44,133,np.inf],[-1,250,450,np.inf],[-1,3.5,9.5,np.inf],[-1,4,5.5,np.inf],[-1,115,175,np.inf],
               [-1,0.35,0.5,np.inf],[-1,82,93,np.inf],[-1,26,38,np.inf],[-1,325,350,np.inf],[-1,12,14,np.inf],[-1,100,350,np.inf],
               [-1,6,11.5,np.inf],[-1,10,17,np.inf],[-1,0.11,0.23,np.inf],[-1,55,70,np.inf],[-1,20,40,np.inf],[-1,3,8,np.inf],
                [-1,1,5,np.inf],[-1,1,np.inf]]
cut_features = list(combined.drop(['乙肝表面抗原','乙肝表面抗体','乙肝e抗原','乙肝e抗体','乙肝核心抗体','性别','年龄','体检日期'],
                                  axis=1).columns)
combined_cut = pd.DataFrame()

for i,feature in enumerate(cut_features):
#     print(i)
    cut_inter = cut_interval[i]
    cut_num = [-1,0,1]
    if len(cut_inter) < 4:
        cut_num = [0,1]
    s = pd.Series(pd.cut(combined_copy[feature],bins=cut_inter,labels=cut_num),name='cut_'+feature)
    combined_cut = pd.concat([combined_cut,s],axis=1)
combined_cut.index = combined.index

In [37]:
# rank特征# 数值型数据 # 两两相乘
df_dtypes = combined_copy.dtypes
float_features =list(df_dtypes[df_dtypes!='object'].index)
combined_rank = pd.DataFrame(index=combined_copy.index)
combined_ratio = pd.DataFrame(index=combined_copy.index)
combined_mul = pd.DataFrame(index=combined_copy.index)
for feature in float_features:
    combined_rank['r'+feature] = combined_copy[feature].rank(method='max')/float(len(combined_copy))
# 两两相除的特征
# 两两相乘的特征
for i,feature1 in enumerate(float_features):
    if len(float_features[i+1:])>0:
        for feature2 in float_features[i+1:]:
            combined_ratio[feature1 + '_' + feature2] = combined_copy[feature1]/combined_copy[feature2]
    for feature2 in float_features[i:]:
            combined_mul[feature1 + '*' + feature2] = combined_copy[feature1]*combined_copy[feature2]
combined_ratio.fillna(combined_ratio.median(axis=0),inplace=True)
combined_ratio[combined_ratio == np.inf] = 1e6
# print(combined_ratio.head())
combined_mul.head()

,年龄*年龄,年龄**天门冬氨酸氨基转换酶,年龄**丙氨酸氨基转换酶,年龄**碱性磷酸酶,年龄**r-谷氨酰基转换酶,年龄**总蛋白,年龄*白蛋白,年龄**球蛋白,年龄*白球比例,年龄*甘油三酯,年龄*总胆固醇,年龄*高密度脂蛋白胆固醇,年龄*低密度脂蛋白胆固醇,年龄*尿素,年龄*肌酐,年龄*尿酸,年龄*乙肝表面抗原,年龄*乙肝表面抗体,年龄*乙肝e抗原,年龄*乙肝e抗体,年龄*乙肝核心抗体,年龄*白细胞计数,年龄*红细胞计数,年龄*血红蛋白,年龄*红细胞压积,年龄*红细胞平均体积,年龄*红细胞平均血红蛋白量,年龄*红细胞平均血红蛋白浓度,年龄*红细胞体积分布宽度,年龄*血小板计数,年龄*血小板平均体积,年龄*血小板体积分布宽度,年龄*血小板比积,年龄*中性粒细胞%,年龄*淋巴细胞%,年龄*单核细胞%,年龄*嗜酸细胞%,年龄*嗜碱细胞%,*天门冬氨酸氨基转换酶**天门冬氨酸氨基转换酶,*天门冬氨酸氨基转换酶**丙氨酸氨基转换酶,*天门冬氨酸氨基转换酶**碱性磷酸酶,*天门冬氨酸氨基转换酶**r-谷氨酰基转换酶,*天门冬氨酸氨基转换酶**总蛋白,*天门冬氨酸氨基转换酶*白蛋白,*天门冬氨酸氨基转换酶**球蛋白,*天门冬氨酸氨基转换酶*白球比例,*天门冬氨酸氨基转换酶*甘油三酯,*天门冬氨酸氨基转换酶*总胆固醇,*天门冬氨酸氨基转换酶*高密度脂蛋白胆固醇,*天门冬氨酸氨基转换酶*低密度脂蛋白胆固醇,...,红细胞体积分布宽度*中性粒细胞%,红细胞体积分布宽度*淋巴细胞%,红细胞体积分布宽度*单核细胞%,红细胞体积分布宽度*嗜酸细胞%,红细胞体积分布宽度*嗜碱细胞%,血小板计数*血小板计数,血小板计数*血小板平均体积,血小板计数*血小板体积分布宽度,血小板计数*血小板比积,血小板计数*中性粒细胞%,血小板计数*淋巴细胞%,血小板计数*单核细胞%,血小板计数*嗜酸细胞%,血小板计数*嗜碱细胞%,血小板平均体积*血小板平均体积,血小板平均体积*血小板体积分布宽度,血小板平均体积*血小板比积,血小板平均体积*中性粒细胞%,血小板平均体积*淋巴细胞%,血小板平均体积*单核细胞%,血小板平均体积*嗜酸细胞%,血小板平均体积*嗜碱细胞%,血小板体积分布宽度*血小板体积分布宽度,血小板体积分布宽度*血小板比积,血小板体积分布宽度*中性粒细胞%,血小板体积分布宽度*淋巴细胞%,血小板体积分布宽度*单核细胞%,血小板体积分布宽度*嗜酸细胞%,血小板体积分布宽度*嗜碱细胞%,血小板比积*血小板比积,血小板比积*中性粒细胞%,血小板比积*淋巴细胞%,血小板比积*单核细胞%,血小板比积*嗜酸细胞%,血小板比积*嗜碱细胞%,中性粒细胞%*中性粒细胞%,中性粒细胞%*淋巴细胞%,中性粒细胞%*单核细胞%,中性粒细胞%*嗜酸细胞%,中性粒细胞%*嗜碱细胞%,淋巴细胞%*淋巴细胞%,淋巴细胞%*单核细胞%,淋巴细胞%*嗜酸细胞%,淋巴细胞%*嗜碱细胞%,单核细胞%*单核细胞%,单核细胞%*嗜酸细胞%,单核细胞%*嗜碱细胞%,嗜酸细胞%*嗜酸细胞%,嗜酸细胞%*嗜碱细胞%,嗜碱细胞%*嗜碱细胞%
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1681,1023.36,947.10,4083.19,829.43,3152.08,2033.60,1118.48,74.62,53.71,181.63,56.17,108.65,240.67,3167.25,14324.99,1.64,132.02,1.64,68.47,69.29,218.94,213.61,6810.1,19.639,3767.9,1307.9,14227.0,524.8,6806.0,405.9,713.4,6.724,2218.1,1402.2,266.5,192.7,24.6,623.0016,576.5760,2485.7664,504.9408,1918.9248,1238.0160,680.9088,45.4272,32.6976,110.5728,34.1952,66.1440,...,692.48,437.76,83.20,60.16,7.68,27556.0,1643.4,2888.4,27.224,8980.6,5677.2,1079.0,780.2,99.6,98.01,172.26,1.6236,535.59,338.58,64.35,46.53,5.94,302.76,2.8536,941.34,595.08,113.10,81.78,10.44,0.026896,8.8724,5.6088,1.0660,0.7708,0.0984,2926.81,1850.22,351.65,254.27,32.46,1169.64,222.30,160.74,20.52,42.25,30.55,3.90,22.09,2.82,0.36
2,1681,1007.37,1486.25,2755.61,3239.00,3256.63,1958.16,1298.47,61.91,115.21,166.46,38.13,107.83,215.66,3571.92,19957.98,1.64,132.02,1.64,68.47,69.29,313.65,213.61,6396.0,18.696,3587.5,1225.9,14022.0,549.4,11357.0,377.2,422.3,10.660,2132.0,1504.7,237.8,192.7,32.8,603.6849,890.6625,1651.3497,1941.0300,1951.5951,1173.4632,778.1319,37.1007,69.0417,99.7542,22.8501,64.6191,...,696.80,491.78,77.72,62.98,10.72,76729.0,2548.4,2853.1,72.020,14404.0,10165.9,1606.6,1301.9,221.6,84.64,94.76,2.3920,478.40,337.64,53.36,43.24,7.36,106.09,2.6780,535.60,378.01,59.74,48.41,8.24,0.067600,13.5200,9.5420,1.5080,1.2220,0.2080,2704.00,1908.40,301.60,244.40,41.60,1346.89,212.86,172.49,29.36,33.64,27.26,4.64,22.09,3.76,0.64
3,2116,957.72,700.58,2929.74,1755.82,3966.58,2208.00,1758.58,57.96,45.54,189.98,75.44,92.46,219.42,3596.74,20795.22,0.46,0.92,0.46,63.02,49.22,211.60,218.96,6844.8,20.148,4227.4,1439.8,15640.0,598.0,11086.0,381.8,763.6,9.154,2212.6,1853.8,354.2,147.2,36.8,433.4724,317.0886,1326.0258,794.6994,1795.3086,999.3600,795.9486,26.2332,20.6118,85.9866,34.1448,41.8482,...,625.30,523.90,100.10,41.60,10.40,58081.0,2000.3,4000.6,47.959,11592.1,9712.3,1855.7,771.2,192.8,68.89,137.78,1.6517,399.23,334.49,63.91,26.56,6.64,275.56,3.3034,798.46,668.98,127.82,53.12,13.28,0.039601,9.5719,8.0197,1.5323,0.6368,0.1592,2313.61,1938.43,370.37,153.92,38.48,1624.09,310.31,128.96,32.24,59.29,24.64,6.16,10.24,2.56,0.64
4,484,329.78,232.98,1629.76,444.84,1561.56,968.44,593.12,35.86,23.32,151.58,31.46,110.88,93.06,1352.12,8114.70,0.88,70.84,0.88,36.74,37.18,205.92,94.38,3014.0,8.866,2065.8,701.8,7480.0,277.2,5544.0,226.6,237.6,5.720,917.4,1023.0,147.4,101.2,11.0,224.7001,158.7441,1110.4592,303.0978,1063.9902,659.8598,404.1304,24.4337,15.8894,103.2811,21.4357,75.5496,...,525.42,585.90,84.42,57.96,6.30,63504.0,2595.6,2721.6,65.520,10508.4,11718.0,1688.4,1159.2,126.0,106.09,111.24,2.6780,429.51,478.95

In [10]:
# combined_ratio.max()

In [33]:
# (pd.to_datetime(combined['体检日期']) - parse('2017-10-09')).dt.days

In [38]:
# 判断有无指标
combined['有无蛋白指标'] = np.where(combined['白蛋白'].isnull(),0,1)
combined['有无固醇指标'] = np.where(combined['甘油三酯'].isnull(),0,1)
combined['有无血指标'] = np.where(combined['血红蛋白'].isnull(),0,1)
combined['有无细胞指标'] = np.where(combined['淋巴细胞%'].isnull(),0,1)
combined['有无酶指标'] = np.where(combined['*碱性磷酸酶'].isnull(),0,1)
combined['有无尿指标'] = np.where(combined['尿素'].isnull(),0,1)
combined['有无肝指标'] = np.where(combined['乙肝表面抗原'].isnull(),0,1)
# 统计确实值的数量
combined['缺失值数量'] = combined.apply(lambda x:combined.shape[1]-x.count(),axis=1)
# 对性别热编码
combined['性别'][combined['性别']== '??'] = '女'
combined['性别'] = combined['性别'].map({'男':1, '女':0})
# combined['体检日期'] = (pd.to_datetime(combined['体检日期']) - parse('2017-10-09')).dt.days
combined.head()

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,*球蛋白,白球比例,甘油三酯,总胆固醇,高密度脂蛋白胆固醇,低密度脂蛋白胆固醇,尿素,肌酐,尿酸,乙肝表面抗原,乙肝表面抗体,乙肝e抗原,乙肝e抗体,乙肝核心抗体,白细胞计数,红细胞计数,血红蛋白,红细胞压积,红细胞平均体积,红细胞平均血红蛋白量,红细胞平均血红蛋白浓度,红细胞体积分布宽度,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%,有无蛋白指标,有无固醇指标,有无血指标,有无细胞指标,有无酶指标,有无尿指标,有无肝指标,缺失值数量
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,41,12/10/2017,24.96,23.10,99.59,20.23,76.88,49.60,27.28,1.82,1.31,4.43,1.37,2.65,5.87,77.25,349.39,NaN,NaN,NaN,NaN,NaN,5.34,5.21,166.1,0.479,91.9,31.9,347.0,12.8,166.0,9.9,17.4,0.164,54.1,34.2,6.5,4.7,0.6,1,1,1,1,1,1,0,5
2,1,41,19/10/2017,24.57,36.25,67.21,79.00,79.43,47.76,31.67,1.51,2.81,4.06,0.93,2.63,5.26,87.12,486.78,NaN,NaN,NaN,NaN,NaN,7.65,5.21,156.0,0.456,87.5,29.9,342.0,13.4,277.0,9.2,10.3,0.260,52.0,36.7,5.8,4.7,0.8,1,1,1,1,1,1,0,5
3,1,46,26/10/2017,20.82,15.23,63.69,38.17,86.23,48.00,38.23,1.26,0.99,4.13,1.64,2.01,4.77,78.19,452.07,0.01,0.02,0.01,1.37,1.07,4.60,4.76,148.8,0.438,91.9,31.3,340.0,13.0,241.0,8.3,16.6,0.199,48.1,40.3,7.7,3.2,0.8,1,1,1,1,1,1,1,0
4,0,22,25/10/2017,14.99,10.59,74.08,20.22,70.98,44.02,26.96,1.63,1.06,6.89,1.43,5.04,4.23,61.46,368.85,NaN,NaN,NaN,NaN,NaN,9.36,4.29,137.0,0.403,93.9,31.9,340.0,12.6,252.0,10.3,10.8,0.260,41.7,46.5,6.7,4.6,0.5,1,1,1,1,1,1,0,5
5,0,48,26/10/2017,20.07,14.78,75.79,22.72,78.05,41.83,36.22,1.15,0.97,5.37,1.27,3.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.07,5.15,106.0,0.354,68.7,20.6,299.0,16.6,316.0,11.1,14.0,0.350,56.6,33.1,9.1,0.6,0.6,1,1,1,1,1,0,0,8


In [37]:
# # 按照每天血糖的平均值排序
# day_mean = combined.groupby('体检日期').agg({'血糖':np.mean})
# time = day_mean.sort_values('血糖').index
# time_mapping = dict((v,i) for i,v in enumerate(time))
# combined['体检日期'] = combined['体检日期'].map(time_mapping)

In [39]:
# 部分特征的统计特征
combined['酶_mean'] = combined.loc[:,'*天门冬氨酸氨基转换酶':'*r-谷氨酰基转换酶'].mean(axis=1)
combined['酶_max'] = combined.loc[:,'*天门冬氨酸氨基转换酶':'*r-谷氨酰基转换酶'].max(axis=1)
combined['酶_min'] = combined.loc[:,'*天门冬氨酸氨基转换酶':'*r-谷氨酰基转换酶'].min(axis=1)
combined['酶_std'] = combined.loc[:,'*天门冬氨酸氨基转换酶':'*r-谷氨酰基转换酶'].std(axis=1)
# combined['固醇高低比'] = combined['高密度脂蛋白胆固醇']/combined['低密度脂蛋白胆固醇']
# combined['尿素尿酸比'] = combined['尿酸']/combined['尿素']
# combined['白红细胞和'] = combined['白细胞计数'] + combined['红细胞计数'] 
# combined['白红细胞比'] = combined['白细胞计数'] / combined['红细胞计数']
combined.head()

,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,*球蛋白,白球比例,甘油三酯,总胆固醇,高密度脂蛋白胆固醇,低密度脂蛋白胆固醇,尿素,肌酐,尿酸,乙肝表面抗原,乙肝表面抗体,乙肝e抗原,乙肝e抗体,乙肝核心抗体,白细胞计数,红细胞计数,血红蛋白,红细胞压积,红细胞平均体积,红细胞平均血红蛋白量,红细胞平均血红蛋白浓度,红细胞体积分布宽度,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%,有无蛋白指标,有无固醇指标,有无血指标,有无细胞指标,有无酶指标,有无尿指标,有无肝指标,缺失值数量,酶_mean,酶_max,酶_min,酶_std
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,41,12/10/2017,24.96,23.10,99.59,20.23,76.88,49.60,27.28,1.82,1.31,4.43,1.37,2.65,5.87,77.25,349.39,NaN,NaN,NaN,NaN,NaN,5.34,5.21,166.1,0.479,91.9,31.9,347.0,12.8,166.0,9.9,17.4,0.164,54.1,34.2,6.5,4.7,0.6,1,1,1,1,1,1,0,5,41.9700,99.59,20.23,38.462575
2,1,41,19/10/2017,24.57,36.25,67.21,79.00,79.43,47.76,31.67,1.51,2.81,4.06,0.93,2.63,5.26,87.12,486.78,NaN,NaN,NaN,NaN,NaN,7.65,5.21,156.0,0.456,87.5,29.9,342.0,13.4,277.0,9.2,10.3,0.260,52.0,36.7,5.8,4.7,0.8,1,1,1,1,1,1,0,5,51.7575,79.00,24.57,25.564143
3,1,46,26/10/2017,20.82,15.23,63.69,38.17,86.23,48.00,38.23,1.26,0.99,4.13,1.64,2.01,4.77,78.19,452.07,0.01,0.02,0.01,1.37,1.07,4.60,4.76,148.8,0.438,91.9,31.3,340.0,13.0,241.0,8.3,16.6,0.199,48.1,40.3,7.7,3.2,0.8,1,1,1,1,1,1,1,0,34.4775,63.69,15.23,21.786833
4,0,22,25/10/2017,14.99,10.59,74.08,20.22,70.98,44.02,26.96,1.63,1.06,6.89,1.43,5.04,4.23,61.46,368.85,NaN,NaN,NaN,NaN,NaN,9.36,4.29,137.0,0.403,93.9,31.9,340.0,12.6,252.0,10.3,10.8,0.260,41.7,46.5,6.7,4.6,0.5,1,1,1,1,1,1,0,5,29.9700,74.08,10.59,29.668948
5,0,48,26/10/2017,20.07,14.78,75.79,22.72,78.05,41.83,36.22,1.15,0.97,5.37,1.27,3.65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.07,5.15,106.0,0.354,68.7,20.6,299.0,16.6,316.0,11.1,14.0,0.350,56.6,33.1,9.1,0.6,0.6,1,1,1,1,1,0,0,8,33.3400,75.79,14.78,28.491832


In [40]:
# 缺失值处理
combined.fillna(combined.median(axis=0),inplace=True)
combined.head()

,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,*球蛋白,白球比例,甘油三酯,总胆固醇,高密度脂蛋白胆固醇,低密度脂蛋白胆固醇,尿素,肌酐,尿酸,乙肝表面抗原,乙肝表面抗体,乙肝e抗原,乙肝e抗体,乙肝核心抗体,白细胞计数,红细胞计数,血红蛋白,红细胞压积,红细胞平均体积,红细胞平均血红蛋白量,红细胞平均血红蛋白浓度,红细胞体积分布宽度,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%,有无蛋白指标,有无固醇指标,有无血指标,有无细胞指标,有无酶指标,有无尿指标,有无肝指标,缺失值数量,酶_mean,酶_max,酶_min,酶_std
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,41,12/10/2017,24.96,23.10,99.59,20.23,76.88,49.60,27.28,1.82,1.31,4.43,1.37,2.65,5.87,77.25,349.39,0.04,3.22,0.04,1.67,1.69,5.34,5.21,166.1,0.479,91.9,31.9,347.0,12.8,166.0,9.9,17.4,0.164,54.1,34.2,6.5,4.7,0.6,1,1,1,1,1,1,0,5,41.9700,99.59,20.23,38.462575
2,1,41,19/10/2017,24.57,36.25,67.21,79.00,79.43,47.76,31.67,1.51,2.81,4.06,0.93,2.63,5.26,87.12,486.78,0.04,3.22,0.04,1.67,1.69,7.65,5.21,156.0,0.456,87.5,29.9,342.0,13.4,277.0,9.2,10.3,0.260,52.0,36.7,5.8,4.7,0.8,1,1,1,1,1,1,0,5,51.7575,79.00,24.57,25.564143
3,1,46,26/10/2017,20.82,15.23,63.69,38.17,86.23,48.00,38.23,1.26,0.99,4.13,1.64,2.01,4.77,78.19,452.07,0.01,0.02,0.01,1.37,1.07,4.60,4.76,148.8,0.438,91.9,31.3,340.0,13.0,241.0,8.3,16.6,0.199,48.1,40.3,7.7,3.2,0.8,1,1,1,1,1,1,1,0,34.4775,63.69,15.23,21.786833
4,0,22,25/10/2017,14.99,10.59,74.08,20.22,70.98,44.02,26.96,1.63,1.06,6.89,1.43,5.04,4.23,61.46,368.85,0.04,3.22,0.04,1.67,1.69,9.36,4.29,137.0,0.403,93.9,31.9,340.0,12.6,252.0,10.3,10.8,0.260,41.7,46.5,6.7,4.6,0.5,1,1,1,1,1,1,0,5,29.9700,74.08,10.59,29.668948
5,0,48,26/10/2017,20.07,14.78,75.79,22.72,78.05,41.83,36.22,1.15,0.97,5.37,1.27,3.65,4.86,77.18,341.67,0.04,3.22,0.04,1.67,1.69,5.07,5.15,106.0,0.354,68.7,20.6,299.0,16.6,316.0,11.1,14.0,0.350,56.6,33.1,9.1,0.6,0.6,1,1,1,1,1,0,0,8,33.3400,75.79,14.78,28.491832


In [9]:
# combined_age = combined.loc[:,'*天门冬氨酸氨基转换酶':'嗜碱细胞%'].apply(lambda x : x/combined['年龄'],axis=0)
# combined = pd.concat([combined,combined_age],axis=1)
# combined.head()

In [42]:
# from pylab import mpl
# mpl.rcParams['font.sans-serif'] = ['FangSong'] # 指定默认字体
# mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
# mpl.rcParams['axes.titlesize'] = 16
# train.loc[:,'*天门冬氨酸氨基转换酶':'尿酸'].hist(bins=50,figsize=(40,30))
# plt.show()

In [59]:
# train_copy = train.copy()
# train_copy.dropna(thresh=21,inplace=True,axis=0)
# 1- train_copy.apply(lambda x: x.count(),axis=0)/(train_copy.shape[0])

In [5]:
# train = pd.merge(train,train_test_rank,left_on='id',right_on='id',how='left')
# train.head()

In [41]:
combined = pd.concat([combined,combined_cut,combined_ratio,combined_mul],axis=1)
combined.head()

,性别,年龄,体检日期,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,*球蛋白,白球比例,甘油三酯,总胆固醇,高密度脂蛋白胆固醇,低密度脂蛋白胆固醇,尿素,肌酐,尿酸,乙肝表面抗原,乙肝表面抗体,乙肝e抗原,乙肝e抗体,乙肝核心抗体,白细胞计数,红细胞计数,血红蛋白,红细胞压积,红细胞平均体积,红细胞平均血红蛋白量,红细胞平均血红蛋白浓度,红细胞体积分布宽度,血小板计数,血小板平均体积,血小板体积分布宽度,血小板比积,中性粒细胞%,淋巴细胞%,单核细胞%,嗜酸细胞%,嗜碱细胞%,有无蛋白指标,有无固醇指标,有无血指标,有无细胞指标,有无酶指标,有无尿指标,有无肝指标,缺失值数量,酶_mean,酶_max,...,红细胞体积分布宽度*中性粒细胞%,红细胞体积分布宽度*淋巴细胞%,红细胞体积分布宽度*单核细胞%,红细胞体积分布宽度*嗜酸细胞%,红细胞体积分布宽度*嗜碱细胞%,血小板计数*血小板计数,血小板计数*血小板平均体积,血小板计数*血小板体积分布宽度,血小板计数*血小板比积,血小板计数*中性粒细胞%,血小板计数*淋巴细胞%,血小板计数*单核细胞%,血小板计数*嗜酸细胞%,血小板计数*嗜碱细胞%,血小板平均体积*血小板平均体积,血小板平均体积*血小板体积分布宽度,血小板平均体积*血小板比积,血小板平均体积*中性粒细胞%,血小板平均体积*淋巴细胞%,血小板平均体积*单核细胞%,血小板平均体积*嗜酸细胞%,血小板平均体积*嗜碱细胞%,血小板体积分布宽度*血小板体积分布宽度,血小板体积分布宽度*血小板比积,血小板体积分布宽度*中性粒细胞%,血小板体积分布宽度*淋巴细胞%,血小板体积分布宽度*单核细胞%,血小板体积分布宽度*嗜酸细胞%,血小板体积分布宽度*嗜碱细胞%,血小板比积*血小板比积,血小板比积*中性粒细胞%,血小板比积*淋巴细胞%,血小板比积*单核细胞%,血小板比积*嗜酸细胞%,血小板比积*嗜碱细胞%,中性粒细胞%*中性粒细胞%,中性粒细胞%*淋巴细胞%,中性粒细胞%*单核细胞%,中性粒细胞%*嗜酸细胞%,中性粒细胞%*嗜碱细胞%,淋巴细胞%*淋巴细胞%,淋巴细胞%*单核细胞%,淋巴细胞%*嗜酸细胞%,淋巴细胞%*嗜碱细胞%,单核细胞%*单核细胞%,单核细胞%*嗜酸细胞%,单核细胞%*嗜碱细胞%,嗜酸细胞%*嗜酸细胞%,嗜酸细胞%*嗜碱细胞%,嗜碱细胞%*嗜碱细胞%
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,41,12/10/2017,24.96,23.10,99.59,20.23,76.88,49.60,27.28,1.82,1.31,4.43,1.37,2.65,5.87,77.25,349.39,0.04,3.22,0.04,1.67,1.69,5.34,5.21,166.1,0.479,91.9,31.9,347.0,12.8,166.0,9.9,17.4,0.164,54.1,34.2,6.5,4.7,0.6,1,1,1,1,1,1,0,5,41.9700,99.59,...,692.48,437.76,83.20,60.16,7.68,27556.0,1643.4,2888.4,27.224,8980.6,5677.2,1079.0,780.2,99.6,98.01,172.26,1.6236,535.59,338.58,64.35,46.53,5.94,302.76,2.8536,941.34,595.08,113.10,81.78,10.44,0.026896,8.8724,5.6088,1.0660,0.7708,0.0984,2926.81,1850.22,351.65,254.27,32.46,1169.64,222.30,160.74,20.52,42.25,30.55,3.90,22.09,2.82,0.36
2,1,41,19/10/2017,24.57,36.25,67.21,79.00,79.43,47.76,31.67,1.51,2.81,4.06,0.93,2.63,5.26,87.12,486.78,0.04,3.22,0.04,1.67,1.69,7.65,5.21,156.0,0.456,87.5,29.9,342.0,13.4,277.0,9.2,10.3,0.260,52.0,36.7,5.8,4.7,0.8,1,1,1,1,1,1,0,5,51.7575,79.00,...,696.80,491.78,77.72,62.98,10.72,76729.0,2548.4,2853.1,72.020,14404.0,10165.9,1606.6,1301.9,221.6,84.64,94.76,2.3920,478.40,337.64,53.36,43.24,7.36,106.09,2.6780,535.60,378.01,59.74,48.41,8.24,0.067600,13.5200,9.5420,1.5080,1.2220,0.2080,2704.00,1908.40,301.60,244.40,41.60,1346.89,212.86,172.49,29.36,33.64,27.26,4.64,22.09,3.76,0.64
3,1,46,26/10/2017,20.82,15.23,63.69,38.17,86.23,48.00,38.23,1.26,0.99,4.13,1.64,2.01,4.77,78.19,452.07,0.01,0.02,0.01,1.37,1.07,4.60,4.76,148.8,0.438,91.9,31.3,340.0,13.0,241.0,8.3,16.6,0.199,48.1,40.3,7.7,3.2,0.8,1,1,1,1,1,1,1,0,34.4775,63.69,...,625.30,523.90,100.10,41.60,10.40,58081.0,2000.3,4000.6,47.959,11592.1,9712.3,1855.7,771.2,192.8,68.89,137.78,1.6517,399.23,334.49,63.91,26.56,6.64,275.56,3.3034,798.46,668.98,127.82,53.12,13.28,0.039601,9.5719,8.0197,1.5323,0.6368,0.1592,2313.61,1938.43,370.37,153.92,38.48,1624.09,310.31,128.96,32.24,59.29,24.64,6.16,10.24,2.56,0.64
4,0,22,25/10/2017,14.99,10.59,74.08,20.22,70.98,44.02,26.96,1.63,1.06,6.89,1.43,5.04,4.23,61.46,368.85,0.04,3.22,0.04,1.67,1.69,9.36,4.29,137.0,0.403,93.9,31.9,340.0,12.6,252.0,10.3,10.8,0.260,41.7,46.5,6.7,4.6,0.5,1,1,1,1,1,1,0,5,29.9700,74.08,...,525.42,585.90,84.42,57.96,6.30,63504.0,2595.6,2721.6,65.520,10508.4,11718.0,1688.4,1159.2,126.0,106.09,111.24,2.6780,429.51,478.95,69.01,47.38,5.15,116.64,2.8080,450.36,502.20,72.36,49.68,5.40,0.067600,10.8420,12.0900,1.7420,1.1960,0.1300,1738.89,1939.05,279.39,191.82,20.85,2162.25,311.55,213.90,23.25,44.89,30.82,3.35,21.16,2.30,0.25
5,0,48,26/10/2017,20.07,14.78,75.79,22.72,78.05,41.83,36.22,1.15,0.97,5.37,1.27,3.65,4.86,77.18,341.67,0.04,3.22,0.04,1.67,1.69,5.07,5.15,106.0,0.354,68.7,20.6,299.0,16.6,316.0,11.1,14.0,0.350,56.6,33.1,9.1,0.6,0.6,1,1,1,1,1,0,0,8,33.3400,75.79,...,939.56,549.46,151.06,9.96,9.96,99856.0,3507.6,4424.0,110.600,17885.6,10459.6,2875.6,189.6,189.6,123.21,155.40,3.8850,628.26,367.41,101.01,6.66,6.66,196.00,4.9000,792.40,463.40,127.40,8.40,8.40,0.122500,19.8100,11.5850,3.1850,0.2100,0.2100,3203.56,1873.46,515.06,33.96,33.96,1095.61,301.21,19

In [16]:
# # y = pd.DataFrame(y)
# # pd.concat([y,test_real_y])


(7642, 1528)

In [42]:
# train = train[train['血糖']<=20]

# y = combined['血糖']
# combined.drop(['体检日期'],axis=1,inplace=True)
train = combined.loc[train_index]
test = combined.loc[test_index]
train.to_csv('../train_data.csv',index=True)
test.to_csv('../test_data.csv',index=True)
y = pd.DataFrame(y)
test_real_y.rename(columns={0:'血糖'},inplace=True)
y = pd.concat([y,test_real_y])
y.index = train.index
y.to_csv('../targets.csv',index=True)

In [31]:
y.head()

,血糖
id,
1,6.06
2,5.39
3,5.59
4,4.30
5,5.42


In [16]:
combined_ratio['年龄'] = combined['年龄']
y = pd.DataFrame(y)
y.to_csv('../targets.csv',index=True)
# combined.drop(['血糖'],axis=1,inplace=True)
train = combined_ratio.loc[train_index]
test = combined_ratio.loc[test_index]
train.to_csv('../train_data_ratio.csv',index=True)
test.to_csv('../test_data_ratio.csv',index=True)

In [15]:
y.to_csv('../targets.csv',index=True)
